In [1]:
import pandas as pd
import mysql.connector

# 1. MySQL 연결
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="분석실습"
)
cursor = conn.cursor()

In [2]:
# 2. 파일에서 데이터프레임으로 읽기 (탭 구분자라고 가정)
df_customer = pd.read_csv("실습데이터/customer.txt", sep="\t", dtype=str,  encoding='cp949')
df_customer.shape
#(5369, 8)

(5369, 8)

In [3]:
df_sales = pd.read_csv("실습데이터/sales.txt", sep="\t", dtype=str,  encoding='cp949')
df_sales.shape
#(541909, 8)

(541909, 8)

In [4]:
# 3. CUSTOMER INSERT
for row in df_customer.itertuples(index=False, name=None):
    cursor.execute("""
        INSERT INTO CUSTOMER (mem_no, last_name, first_name, gd, birth_dt, entr_dt, grade, sign_up_ch)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, row)
    
conn.commit()  # 변경사항 저장

cursor.execute("SELECT COUNT(*) FROM CUSTOMER")
print(f"총 {cursor.fetchone()[0]}건 삽입 완료")

총 5369건 삽입 완료


In [5]:
# 4. SALES 데이터전처리
df_sales = df_sales.dropna(subset=["CustomerID"])
df_sales["CustomerID"] = df_sales["CustomerID"].astype(int)

# 날짜 및 숫자 변환
df_sales["Quantity"] = df_sales["Quantity"].astype(int)
df_sales["UnitPrice"] = df_sales["UnitPrice"].astype(float)
df_sales["CustomerID"] = df_sales["CustomerID"].astype(int)

# DATETIME 변환
df_sales["InvoiceDate"] = pd.to_datetime(df_sales["InvoiceDate"], errors='coerce')
df_sales["InvoiceDate"] = df_sales["InvoiceDate"].dt.strftime('%Y-%m-%d %H:%M:%S')
df_sales.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [6]:
# 5. SALES INSERT
df_sales = df_sales.astype(object)
rows = [tuple(x) for x in df_sales.to_records(index=False)]
cursor.executemany("""
    INSERT INTO SALES (InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
""", rows)

conn.commit()

cursor.execute("SELECT COUNT(*) FROM SALES")
print(f"총 {cursor.fetchone()[0]}건 삽입 완료")

총 406829건 삽입 완료


In [7]:
# 6. 종료
cursor.close()
conn.close()
print("데이터 삽입 완료")

데이터 삽입 완료
